<a href="https://colab.research.google.com/github/meghamkpatel/Skunks_Skool/blob/main/RedWineQuality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Science Engineering
##Assignment 2 - Red Wine Quality
*AutoML*
_______________________________________


###About the dataset
to doooooo

Initializing H2O...

In [3]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 177.2 MB 49 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.1-py2.py3-none-any.whl size=177276478 sha256=aed996e0dbb62a8a9ccf7e75266c86eb09bb8b782e3aad97fa006f155c0c96ff
  Stored in directory: /root/.cache/pip/wheels/a1/a1/d9/bb37df368c4635a707e7362d1088450b606041f05aeba5f173
Successfully built h2o


In [24]:
%matplotlib inline
import random, os, sys
import h2o
import pandas
import pprint
import operator
import matplotlib
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from tabulate import tabulate
from h2o.automl import H2OAutoML
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np

In [20]:
min_mem_size=6 
run_time=222

In [34]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

5


In [ ]:
# 65535 Highest port no
port_no=random.randint(5555,55555)

try:
  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
except:
  logging.critical('h2o.init')
  h2o.download_all_logs(dirname=logs_path, filename=logfile)      
  h2o.cluster().shutdown()
  sys.exit(2)

In [5]:
#installing dependencies
!pip install eli5
!pip install -q kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 7.8 MB/s 
     |████████████████████████████████| 133 kB 51.1 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=49b3ad502fea85aaa38e1460976d4d9a1d09741d42f19d2ed967d4591a8aaeb7
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [6]:
#making a directory for kaggle api
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [10]:
#downloading and reading the red wine dataset
! kaggle datasets list -s 'red wine quality' # searching through kaggle with keyword 'red wine'
! kaggle datasets download -d uciml/red-wine-quality-cortez-et-al-2009
!unzip red-wine-quality-cortez-et-al-2009.zip

ref                                                title                                      size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------  ----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
uciml/red-wine-quality-cortez-et-al-2009           Red Wine Quality                           26KB  2017-11-27 23:41:08         152053       2225  0.88235295       
ruthgn/wine-quality-data-set-red-white-wine        Wine Quality Data Set (Red & White Wine)   98KB  2021-11-03 01:26:46           1447         38  1.0              
yasserh/wine-quality-dataset                       Wine Quality Dataset                       21KB  2022-01-15 19:15:11          18369        454  1.0              
pritsheta/redwinequality                           RED-WINE-QUALITY                           26KB  2021-08-30 05:58:27            122         18  0.9705882        
piyushgoya

In [ ]:
h2o.init(strict_version_check=False) # start h2o

In [11]:
# set this to True if interactive (matplotlib) plots are desired
interactive = True
if not interactive: matplotlib.use('Agg', warn=False)
import matplotlib.pyplot as plt

In [17]:
#Viewing data in the file 
data = h2o.import_file("winequality-red.csv")
data.shape
data.describe()
data.head()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:1599
Cols:12

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,real,real,real,real,real,real,real,real,real,real,real,int
mins,4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4,3.0
mean,8.31963727329581,0.5278205128205127,0.2709756097560977,2.53880550343965,0.08746654158849282,15.874921826141343,46.46779237023139,0.996746679174484,3.3111131957473416,0.6581488430268919,10.422983114446529,5.636022514071294
maxs,15.9,1.58,1.0,15.5,0.611,72.0,289.0,1.00369,4.01,2.0,14.9,8.0
sigma,1.7410963181276957,0.17905970415353537,0.1948011374053186,1.4099280595072798,0.0470653020100901,10.460156969809725,32.895324478299074,0.0018873339538425153,0.1543864649035426,0.16950697959010994,1.0656675818473942,0.8075694397347049
zeros,0,0,132,0,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5.0


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,6
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5
7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4,5
7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10,7
7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,7
7.5,0.5,0.36,6.1,0.071,17,102,0.9978,3.35,0.8,10.5,5


In [18]:
# assign target and inputs for logistic regression
y = 'quality'
X = [name for name in data.columns if name != y]
print(y)
print(X)

quality
['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


##Regression

In [21]:
aml = H2OAutoML(max_runtime_secs=run_time,exclude_algos = ['DeepLearning'])

In [26]:
model_start_time = time.time()
  
try:
  aml.train(x=X,y=y,training_frame=data)  # Change training_frame=train
except Exception as e:
  logging.critical('aml.train') 
  h2o.download_all_logs(dirname=logs_path, filename=logfile)      
  h2o.cluster().shutdown()   
  sys.exit(4)

AutoML progress: |██████████████████████████████████████████████████████████
21:31:51.727: GBM_lr_annealing_selection_AutoML_1_20221023_212824 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

█████| (done) 100%


In [28]:
meta_data={}
meta_data['model_execution_time'] = {"regression":(time.time() - model_start_time)} 
meta_data

{'model_execution_time': {'regression': 290.86050939559937}}

In [29]:
print(aml.leaderboard)

model_id                                                     rmse       mse       mae      rmsle    mean_residual_deviance
StackedEnsemble_BestOfFamily_6_AutoML_1_20221023_212824  0.555415  0.308485  0.382896  0.0865087                  0.308485
StackedEnsemble_BestOfFamily_4_AutoML_1_20221023_212824  0.555759  0.308868  0.37351   0.0864335                  0.308868
StackedEnsemble_AllModels_3_AutoML_1_20221023_212824     0.555879  0.309001  0.389381  0.0866595                  0.309001
GBM_grid_1_AutoML_1_20221023_212824_model_45             0.563121  0.317105  0.374892  0.087544                   0.317105
StackedEnsemble_AllModels_2_AutoML_1_20221023_212824     0.564097  0.318206  0.396641  0.0879426                  0.318206
StackedEnsemble_BestOfFamily_2_AutoML_1_20221023_212824  0.564164  0.318282  0.397581  0.087947                   0.318282
StackedEnsemble_AllModels_1_AutoML_1_20221023_212824     0.564618  0.318794  0.397813  0.0880014                  0.318794
StackedEnsemble_

Analyzing Best Model

In [31]:
best_model = h2o.get_model(aml.leaderboard[2,'model_id'])
best_model.algo

'stackedensemble'

In [32]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", category = matplotlib.cbook.mplDeprecation)

In [35]:
best_model.varimp_plot()

H2OValueError: ignored